<a href="https://www.kaggle.com/code/zittagustavo/neural-network-handmade-from-scratch?scriptVersionId=123048397" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Algoritimo de rede neural feito sem framework

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
  def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

In [3]:
class network(object):
  #size= numero de neuronios nas respectivas camadas
  def __init__(self, sizes):
    self.num_layers = len(sizes) #tamanho ou numero de camadas
    self.sizes = sizes #alocando a variavel para self
    self.biases = [np.random.randn(y,1) for y in sizes[1:]] #setando o bias com um valor aleatorio da distribuição gaussiana(zero de media e desvio padrao de 1) #setando o bias a partir do primeiro(pois é uma camada de entrada e nao precisa de bias)
    self.weights = [np.random.randn(y,x) for x, y in zip(sizes[:-1], sizes[1:])]  #setando o peso com um valor aleatorio da distribuição gaussiana(zero de media e desvio padrao de 1) 

  def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))


  #o metodo feedforward aplica aplica toda a função que fizemos acima(sigmoid) para cada camada
  def feedforward(self,a):
    for b,w in zip(self.biases, self.weights):#rodando um loop para bias e pesos
      a = sigmoid(np.dot(w,a)+b) #aplicando a sigmoid para cada camada de peso e bias
      return a

  def SGD(self, training_data ,epochs, mini_batch_size, eta, test_data=None):

    #####################sumario#######################
    #training_data=lista de tuplas com entradas de treinamento com as saidas desejadas(targets)
    #epochs=numero de epocas para treinar
    #mini_batch_size= tamanho dos mini lotes a serem usados durante a amostragem
    #eta= taxa de aprendizagem(learning rate ou LR)
    #se o teste_data for fornecido, o programa avaliara a rede apos cada periodo de treinamento e vai dar um print nas parciais



    training_data=list(training_data) #lista de tuplas
    n = len(training_data) #tamanho dessa lista de tuplas

    if test_data:#se existir test_data
      test_data=list(test_data) #vai criar uma lista de tuplas
      n_test= len(test_data) #e tambem vai criar o tamanho desta lista de tuplas

    for j in range(epochs):
    #neste loop estamos andando o numero de epocas e dividindo nossas amostras em batches
    #estamos embaralhando, percorrendo a amostra com o numero de passos(mini_batch_size) e separando essa amostra com base nisso

      random.shuffle(training_data)#embaralha os dados(amostra)
      mini_batches = [training_data[k:k+mini_batch_size]for k in range(0,n,mini_batch_size)] #separa o tamanho conforme o mini_batch_size

      for mini_batch in mini_batches:
        #para cada mini_batch aplicamos um unico passo de descida do gradiente
        self.update_mini_batch(mini_batch, eta)#essa função atualiza os pesos e bias

      if test_data:#se passar o teste data vai dar o print
        print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test));
      else:
        print("Epoch {} finalizada".format(j))

  def update_mini_batch(self, mini_batch,eta):
    nabla_b = [np.zeros(b.shape) for b in self.biases]#vai retornar um array cheio de 0 
    nabla_w = [np.zeros(w.shape) for w in self.weights]#vai retornar um array cheio de 0 

    for x,y in mini_batch:
      #o core do update é essa linha
      #usa um algoritmo de backpropagation(calcula os gradientes para cada exemplo de treinamento no mini_batch)
      delta_nabla_b, delta_nabla_w = self.backprop(x,y)
      nabla_b = [nb+dnb for nb, dnd in zip(nabla_b, delta_nabla_b)] #atualiza os biases
      nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)] #atualiza os pesos

    self.weights = [w-(eta/len(mini_batch))*nw for w,nw in zip(self.weights, nabla_w)]
    self.biases  = [b-(eta/len(mini_batch))*nb for b,nb in zip(self.biases, nabla_b)]
    
  
  def backprop(self,x,y):
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]

    #feed forward
    activation = x

    #lista para armazenar todas as ativaçoes, camada por camada
    activations = [x]

    #lista para armazenar todos os vetoresz, camada por camada
    zs = []


    for b,w in zip(self.biases, self.weights):
      z = np.dot(w,activation)+b
      zs.append(z)
      activation = sigmoid(z)
      activations.append(activation)

    #Backward Pass
    delta = self.cost_derivative(activations[-1],y)*sigmoid_prime(zs[-1])
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())

    for l in range(2, self.num_layers):
      z = zs[-l]
      sp = sigmoid_prime(z)
      delta = np.dot(self.weights[-l+1].transpose(),delta)*sp
      nabla_b[-l] = delta
      nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
    return (nabla_b,nabla_w)
    



